In [1]:
import math
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq
from peft import PeftMixedModel, LoraConfig
from datasets import load_dataset
from typing import Optional, Dict, Any

from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载模型和tokenizer
base_model_name = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False)

class CombinedMedicalDataset(Dataset):
    def __init__(
        self,
        tokenizer,
        dataset_name: str,
        question_field: str,
        answer_field: str,
        max_length: int = 512,
        split: str = "train",
        subset_name: Optional[str] = None
    ):
        if subset_name:
            self.dataset = load_dataset(dataset_name, name=subset_name, split=split)
        else:
            self.dataset = load_dataset(dataset_name, split=split)

        self.tokenizer = tokenizer
        self.max_length = max_length
        self.question_field = question_field
        self.answer_field = answer_field

        self.template = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
                        "<|im_start|>user\n{question}<|im_end|>\n"
                        "<|im_start|>assistant\n{answer}<|im_end|>\n")

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx) -> Dict[str, Any]:
        item = self.dataset[idx]

        question = item[self.question_field]
        answer = item[self.answer_field]

        # 构造问题部分（包括system和user消息）
        prompt_text = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
                      "<|im_start|>user\n{}<|im_end|>\n"
                      "<|im_start|>assistant\n").format(question)

        # 构造完整文本
        full_text = self.template.format(question=question, answer=answer)

        # 编码完整文本，但不做padding
        encodings = self.tokenizer(
            full_text,
            truncation=True,
            max_length=self.max_length,
            return_tensors=None  # 返回列表而不是tensor
        )

        # 获取prompt部分的长度
        prompt_tokens = self.tokenizer(
            prompt_text,
            truncation=True,
            max_length=self.max_length
        )
        prompt_length = len(prompt_tokens['input_ids'])

        # 获取序列实际长度
        sequence_length = len(encodings['input_ids'])

        # 创建attention mask和labels
        attention_mask = [1] * prompt_length + [1] * (sequence_length - prompt_length)
        labels = [-100] * prompt_length + encodings['input_ids'][prompt_length:]

        return {
            'input_ids': encodings['input_ids'],
            'attention_mask': attention_mask,
            'labels': labels,
        }

def collate_fn(batch):
    """
    处理变长序列的批处理函数，使用模型的pad_token进行padding
    """
    # 找出最长序列的长度
    max_length = max([len(x['input_ids']) for x in batch]) + 1

    input_ids = []
    attention_mask = []
    labels = []

    # 对每个样本进行padding
    for item in batch:
        # 计算需要的padding长度
        padding_length = max_length - len(item['input_ids'])

        # Padding input_ids，使用pad_token_id
        padded_input_ids = item['input_ids'] + [tokenizer.pad_token_id] * padding_length
        input_ids.append(padded_input_ids)

        # Padding attention_mask，对应位置为0
        padded_attention_mask = item['attention_mask'][1:] + [0] * (padding_length + 1)
        attention_mask.append(padded_attention_mask)

        # Padding labels，对应位置为-100（不计算损失）
        padded_labels = item['labels'][1:] + [-100] * (padding_length + 1)
        labels.append(padded_labels)

    return {
        'input_ids': torch.tensor(input_ids),
        'attention_mask': torch.tensor(attention_mask),
        'labels': torch.tensor(labels)
    }

def setup_training(batch_size=8):
    # 创建数据集
    task1_dataset = CombinedMedicalDataset(
        tokenizer=tokenizer,
        dataset_name="fzkuji/cMedQA2",
        question_field="question",
        answer_field="answer",
        max_length=1024,
        split="train",
        subset_name="deduplicate_all"
    )

    task2_dataset = CombinedMedicalDataset(
        tokenizer=tokenizer,
        dataset_name="fzkuji/HealthCareMagic-100k",
        question_field="input",
        answer_field="output",
        max_length=1024,
        split="train"
    )

    # 创建数据加载器
    task1_loader = DataLoader(
        task1_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    task2_loader = DataLoader(
        task2_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

    return {
        'tokenizer': tokenizer,
        'task1_loader': task1_loader,
        'task2_loader': task2_loader,
    }

In [2]:
# 获取训练设置
training_setup = setup_training(batch_size=8)
task1_loader = training_setup['task1_loader']
task2_loader = training_setup['task2_loader']

# 创建数据迭代器
task1_iter = iter(task1_loader)
task2_iter = iter(task2_loader)

Using the latest cached version of the dataset since fzkuji/cMedQA2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'deduplicate_all' at /root/.cache/huggingface/datasets/fzkuji___c_med_qa2/deduplicate_all/0.0.0/2aa94e762ffb3d6cf5d906d1d47d7579b228e2ce (last modified on Sat Dec  7 21:38:51 2024).
Using the latest cached version of the dataset since fzkuji/HealthCareMagic-100k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/fzkuji___health_care_magic-100k/default/0.0.0/b2db1a6e13bb0e6cf0077375c50e55639b2aeb39 (last modified on Sat Dec  7 21:39:20 2024).


In [3]:
# # 获取一个批次示例
# batch = next(task1_iter)
# print("Batch shapes:")
# for k, v in batch.items():
#     print(f"{k}: {v.shape}")

# print(batch['input_ids'][0])
# print(batch['labels'][0])

# from torch.cuda.amp import autocast, GradScaler

# def compute_loss(logits, labels):
#     """
#     计算损失函数
#     Args:
#         logits: (batch_size, sequence_length, vocab_size)
#         labels: (batch_size, sequence_length) with shifted labels
#     """
#     loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
#     flat_logits = logits.view(-1, logits.size(-1))
#     flat_labels = labels.view(-1)

#     loss = loss_fct(flat_logits, flat_labels)
    
#     return loss

# mixed_model = AutoModelForCausalLM.from_pretrained(base_model_name)


# mixed_model.train()
# mixed_model.to(device)

# outputs = mixed_model(
#     input_ids=batch['input_ids'].to(device),
#     attention_mask=batch['attention_mask'].to(device),
#     return_dict=True
# )
# loss = compute_loss(outputs.logits, batch['labels'].to(device))
# loss.item()

In [4]:
model = AutoModelForCausalLM.from_pretrained(base_model_name)

# 定义LoRA配置
peft_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)

# 创建PeftMixedModel
mixed_model = PeftMixedModel(model, peft_config, adapter_name="0")
mixed_model.add_adapter("1", peft_config)
mixed_model.set_adapter(["0", "1"])

mixed_model.to(device)

# 训练参数
train_config = {
    'train_steps': 1000,
    'gradient_accumulation_steps': 8,
    'lr_start': 1e-4,
    'lr_end': 0,
    'warmup_steps': 100,
    'initial_lr': 0.0
}

# 创建优化器
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, mixed_model.parameters()),
    lr=train_config['initial_lr']
)

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def compute_loss(logits, labels):
    """
    计算损失函数
    Args:
        logits: (batch_size, sequence_length, vocab_size)
        labels: (batch_size, sequence_length) with shifted labels
    """
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    flat_logits = logits.view(-1, logits.size(-1))
    flat_labels = labels.view(-1)
    loss = loss_fct(flat_logits, flat_labels)
    return loss

# 保存第0层 lora_A 第0个 LoRA 参数的初始值
lora_param_name = "base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight"
initial_lora_param = mixed_model.state_dict()[lora_param_name].clone()

# 创建 GradScaler
scaler = GradScaler()

# 初始化计数器
accumulation_loss = 0

# 设置模型为训练模式
mixed_model.train()

# 开始训练
for step in range(train_config['train_steps']):
    # 四种情况轮换(1-4)
    current_task = (step % 4) + 1

    # 冻结所有LoRA参数
    for n, p in mixed_model.named_parameters():
        if "lora" in n:
            p.requires_grad = False

    # 根据任务设置adapter和可训练参数
    if current_task == 1:
        mixed_model.set_adapter("0")
        for n, p in mixed_model.named_parameters():
            p.requires_grad = True if "0.weight" in n else False
    elif current_task == 2:
        mixed_model.set_adapter(["0", "1"])
        for n, p in mixed_model.named_parameters():
            p.requires_grad = True if "0.weight" in n else False
    elif current_task == 3:
        mixed_model.set_adapter("1")
        for n, p in mixed_model.named_parameters():
            p.requires_grad = True if "1.weight" in n else False
    else:  # current_task == 4
        mixed_model.set_adapter(["0", "1"])
        for n, p in mixed_model.named_parameters():
            p.requires_grad = True if "1.weight" in n else False

    for i in range(train_config['gradient_accumulation_steps']):

        # 根据任务选择数据集(1,2用task1的数据，3,4用task2的数据)
        if current_task <= 2:
            try:
                batch = next(task1_iter)
            except StopIteration:
                task1_iter = iter(task1_loader)
                batch = next(task1_iter)
        else:
            try:
                batch = next(task2_iter)
            except StopIteration:
                task2_iter = iter(task2_loader)
                batch = next(task2_iter)

        batch = {k: v.to(device) for k, v in batch.items()}

        # 动态调节学习率
        if step < train_config['warmup_steps']:
            current_lr = (step / train_config['warmup_steps']) * train_config['lr_start']
        else:
            t = (step - train_config['warmup_steps']) / (train_config['train_steps'] - train_config['warmup_steps'])
            current_lr = train_config['lr_end'] + (train_config['lr_start'] - train_config['lr_end']) * math.cos(t * math.pi / 2)

        for param_group in optimizer.param_groups:
            param_group['lr'] = current_lr

        # 前向传播和损失计算
        with autocast():
            outputs = mixed_model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                return_dict=True
            )
            loss = compute_loss(outputs.logits, batch['labels'])

        # 累积梯度
        loss = loss / train_config['gradient_accumulation_steps']
        accumulation_loss += loss.item()
        scaler.scale(loss).backward()

    # if accumulation_count % train_config['gradient_accumulation_steps'] == 0:
    # 执行优化器步骤
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(mixed_model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()

    if step % 10 == 0:
        # 每次执行优化器步骤后，检查参数变化
        updated_lora_param = mixed_model.state_dict()[lora_param_name]
        # if not torch.equal(initial_lora_param, updated_lora_param):
        #     print(f"[Step {step}] Parameter {lora_param_name} has changed.")
        # else:
        #     print(f"[Step {step}] Parameter {lora_param_name} has NOT changed.")

        # 更新初始值为当前值
        initial_lora_param = updated_lora_param.clone()

        print(f"Global step: {step}, Task: {current_task}, Data: {'task1' if current_task <= 2 else 'task2'}, "
              f"Adapter: {'0' if current_task == 1 else '1' if current_task == 3 else '0+1'}, "
              f"loss: {accumulation_loss:.4f}, lr: {current_lr:.6e}")
    accumulation_loss = 0

print("Training completed.")


Global step: 0, Task: 1, Data: task1, Adapter: 0, loss: 3.0251, lr: 0.000000e+00
Global step: 10, Task: 3, Data: task2, Adapter: 1, loss: 2.7623, lr: 1.000000e-05
Global step: 20, Task: 1, Data: task1, Adapter: 0, loss: 2.9234, lr: 2.000000e-05
Global step: 30, Task: 3, Data: task2, Adapter: 1, loss: 2.6990, lr: 3.000000e-05
Global step: 40, Task: 1, Data: task1, Adapter: 0, loss: 3.0030, lr: 4.000000e-05
Global step: 50, Task: 3, Data: task2, Adapter: 1, loss: 2.6993, lr: 5.000000e-05
Global step: 60, Task: 1, Data: task1, Adapter: 0, loss: 2.9024, lr: 6.000000e-05
Global step: 70, Task: 3, Data: task2, Adapter: 1, loss: 2.5661, lr: 7.000000e-05
Global step: 80, Task: 1, Data: task1, Adapter: 0, loss: 2.7241, lr: 8.000000e-05
Global step: 90, Task: 3, Data: task2, Adapter: 1, loss: 2.6528, lr: 9.000000e-05
Global step: 100, Task: 1, Data: task1, Adapter: 0, loss: 2.7619, lr: 1.000000e-04
Global step: 110, Task: 3, Data: task2, Adapter: 1, loss: 2.4413, lr: 9.998477e-05
Global step: 12

In [7]:
def setup_test_data(batch_size=1):
    """设置测试数据加载器"""
    task1_test_dataset = CombinedMedicalDataset(
        tokenizer=tokenizer,
        dataset_name="fzkuji/cMedQA2",
        question_field="question",
        answer_field="answer",
        max_length=1024,  # 保持与训练时相同
        split="test",  # 使用测试集
        subset_name="deduplicate_all"
    )

    task2_test_dataset = CombinedMedicalDataset(
        tokenizer=tokenizer,
        dataset_name="fzkuji/HealthCareMagic-100k",
        question_field="input",
        answer_field="output",
        max_length=1024,
        split="train[:1%]"
    )

    task1_test_loader = DataLoader(
        task1_test_dataset,
        batch_size=batch_size,
        shuffle=False,  # 测试时不需要打乱
        collate_fn=collate_fn
    )
    
    task2_test_loader = DataLoader(
        task2_test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    return task1_test_loader, task2_test_loader

def evaluate(model, test_loader, adapter_names, device, max_test_steps=None):
    """评估函数"""
    model.eval()
    total_loss = 0
    total_steps = 0

    if adapter_names is not None:
        # 设置adapter
        model.set_adapter(adapter_names)
    else:
        model.set_adapter([])
    
    with torch.no_grad():
        for batch in test_loader:
            if max_test_steps and total_steps >= max_test_steps:
                break
                
            batch = {k: v.to(device) for k, v in batch.items()}
            
            with autocast():
                outputs = model(
                    input_ids=batch['input_ids'],
                    # attention_mask=batch['attention_mask'],
                    return_dict=True
                )
                loss = compute_loss(outputs.logits, batch['labels'])
            
            total_loss += loss.item()
            total_steps += 1
            
            if total_steps % 10 == 0:
                print(f"Evaluated {total_steps} steps, current avg loss: {total_loss/total_steps:.4f}")
    
    return total_loss / total_steps

def run_evaluation(max_test_steps=100):
    """运行评估"""
    print("开始加载测试数据...")
    task1_test_loader, task2_test_loader = setup_test_data(batch_size=1)
    
    # 定义测试配置
    test_configs = [
        {
            "name": "Task1 without LoRA",
            "loader": task1_test_loader,
            "adapter": None
        },
        {
            "name": "Task1 with LoRA0 only",
            "loader": task1_test_loader,
            "adapter": "0"
        },
        # {
        #     "name": "Task1 with LoRA1 only",
        #     "loader": task1_test_loader,
        #     "adapter": ["1"]
        # },
        {
            "name": "Task1 with both LoRAs",
            "loader": task1_test_loader,
            "adapter": ["0", "1"]
        },
        {
            "name": "Task2 without LoRA",
            "loader": task2_test_loader,
            "adapter": None
        },
        # {
        #     "name": "Task2 with LoRA0 only",
        #     "loader": task2_test_loader,
        #     "adapter": "0"
        # },
        {
            "name": "Task2 with LoRA1 only",
            "loader": task2_test_loader,
            "adapter": "1"
        },
        {
            "name": "Task2 with both LoRAs",
            "loader": task2_test_loader,
            "adapter": ["0", "1"]
        }
    ]
    
    results = []
    
    # 执行评估
    for config in test_configs:
        print(f"\n评估 {config['name']}...")
        loss = evaluate(
            model=mixed_model,
            test_loader=config['loader'],
            adapter_names=config['adapter'],
            device=device,
            max_test_steps=max_test_steps
        )
        
        results.append({
            "test_name": config['name'],
            "loss": loss
        })
        print(f"{config['name']} - 平均损失: {loss:.4f}")
    
    # 打印总结果
    print("\n评估结果总结:")
    print("-" * 50)
    for result in results:
        print(f"{result['test_name']}: {result['loss']:.4f}")
    
    return results

# 如果已经训练好了模型，可以加载保存的权重
def load_trained_adapters(model, adapter0_path, adapter1_path):
    """加载训练好的adapter权重"""
    model.load_adapter(adapter0_path, adapter_name="0")
    model.load_adapter(adapter1_path, adapter_name="1")
    return model

# 生成示例函数
def generate_example(model, input_text, adapter_names, max_new_tokens=100):
    """使用模型生成回答"""
    # 准备输入
    prompt_text = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
                  "<|im_start|>user\n{}<|im_end|>\n"
                  "<|im_start|>assistant\n").format(input_text)
    
    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
    
    # 设置adapter
    model.set_adapter(adapter_names)
    model.eval()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

    # 1. 如果需要加载已训练的权重：
    # mixed_model = load_trained_adapters(
    #     mixed_model,
    #     adapter0_path="path/to/adapter0",
    #     adapter1_path="path/to/adapter1"
    # )
    
# 2. 运行评估
print("开始评估...")
results = run_evaluation(max_test_steps=100)

# 3. 生成一些示例回答
print("\n生成示例回答:")
test_questions = [
    "我最近经常感觉胸口疼痛，这是怎么回事？",
    "What should I do if I have a persistent headache?",
]

for question in test_questions:
    print("\n问题:", question)
    
    print("\n使用LoRA0的回答:")
    response = generate_example(mixed_model, question, "0")
    print(response)
    
    print("\n使用LoRA1的回答:")
    response = generate_example(mixed_model, question, "1")
    print(response)
    
    print("\n使用两个LoRA的回答:")
    response = generate_example(mixed_model, question, ["0", "1"])
    print(response)

开始评估...
开始加载测试数据...


Using the latest cached version of the dataset since fzkuji/cMedQA2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'deduplicate_all' at /root/.cache/huggingface/datasets/fzkuji___c_med_qa2/deduplicate_all/0.0.0/2aa94e762ffb3d6cf5d906d1d47d7579b228e2ce (last modified on Sat Dec  7 21:38:51 2024).
Using the latest cached version of the dataset since fzkuji/HealthCareMagic-100k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/fzkuji___health_care_magic-100k/default/0.0.0/b2db1a6e13bb0e6cf0077375c50e55639b2aeb39 (last modified on Sat Dec  7 21:39:20 2024).



评估 Task1 without LoRA...
Evaluated 10 steps, current avg loss: 2.9604
Evaluated 20 steps, current avg loss: 3.2198
Evaluated 30 steps, current avg loss: 3.0665
Evaluated 40 steps, current avg loss: 3.1632
Evaluated 50 steps, current avg loss: 3.1793
Evaluated 60 steps, current avg loss: 3.2678
Evaluated 70 steps, current avg loss: 3.3414
Evaluated 80 steps, current avg loss: 3.2962
Evaluated 90 steps, current avg loss: 3.2905
Evaluated 100 steps, current avg loss: 3.2729
Task1 without LoRA - 平均损失: 3.2729

评估 Task1 with LoRA0 only...
Evaluated 10 steps, current avg loss: 2.6060
Evaluated 20 steps, current avg loss: 2.8086
Evaluated 30 steps, current avg loss: 2.6910
Evaluated 40 steps, current avg loss: 2.7817
Evaluated 50 steps, current avg loss: 2.7937
Evaluated 60 steps, current avg loss: 2.8220
Evaluated 70 steps, current avg loss: 2.8820
Evaluated 80 steps, current avg loss: 2.8372
Evaluated 90 steps, current avg loss: 2.8460
Evaluated 100 steps, current avg loss: 2.8299
Task1 wit